# Alternating K Groups

This notebook contains the experiment to investigate the question **What is the benefit of restricting alternating random groups?**

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [1]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [2]:
import numpy as np
import pickle
import multiprocessing

In [3]:
from lib.measures import *
from lib.experiment import run_experiment, save_summary, load_summary
from lib.settings.calibration_settings import settings_lockdown_dates

## 1. General settings  

#### Set the random seed for reproducibility

In [4]:
# Choose random seed
c = 0
np.random.seed(c)
TO_HOURS = 24.0

# Define prefix string used to save plots
expstr = 'k-groups'

#### Set the number of roll-outs to simulate

In [5]:
random_repeats = 1 # Set to at least 20 to obtain stable results

#### Set the time to simulate

In [6]:
end_date = '2020-07-31'

#### Define locations, mobility settings used for simulation, and experiment parameters

In [9]:
locs = {
#     'GER': {
#         'TU': 'lib/mobility/Tubingen_settings_10.pk',
#         'KL': 'lib/mobility/Kaiserslautern_settings_10.pk',
#         'RH': 'lib/mobility/Ruedesheim_settings_10.pk', 
#         'TR': 'lib/mobility/Tirschenreuth_settings_10.pk', 
#     },
    'CH': {
        'VD': 'lib/mobility/Lausanne_settings_10.pk', 
        'LU': 'lib/mobility/Lucerne_settings_5.pk', 
        'TI': 'lib/mobility/Locarno_settings_2.pk', 
        'JU': 'lib/mobility/Jura_settings_10.pk',
    }
}

heuristic_seeds = True
dry_run = True

K_groups = [2, 3, 4]
K_groups_weeks = 6

## 2. Run the simulations

Use settings as above and simulate in the future with additional measures

**WARNING: the following cells might take a long time to run depending of the parameters defined above!**

In [28]:
for country, areas in locs.items():
    for area, mob_settings in areas.items():
        s = []
        
        # start simulation when lockdown ends
#         start_date = settings_lockdown_dates[country]['end']
        start_date = '2020-03-10'

        # baseline
        baseline = run_experiment(
            country=country, 
            area=area, 
            mob_settings=mob_settings,
            start_date=start_date, 
            end_date=end_date, 
            measure_list=[], 
            random_repeats=random_repeats,
            test_update=None, 
            heuristic_seeds=heuristic_seeds,
            dry_run=dry_run)

        # experiment
        for j, k in enumerate(K_groups):
            
            # additional measures
            m = [
                SocialDistancingForKGroups(
                 t_window=Interval(0.0, (24.0 * 7 * K_groups_weeks)), 
                 K=k),
            ] 
            
            # update testing params
            test_update = None
            
            # run
            res = run_experiment(
                country=country, 
                area=area, 
                mob_settings=mob_settings,
                start_date=start_date, 
                end_date=end_date, 
                measure_list=[], 
                random_repeats=random_repeats,
                test_update=None, 
                heuristic_seeds=heuristic_seeds,
                dry_run=dry_run)
            
            s.append(res)
            
            print(country, area, k, ' done.', flush=True)
        
        # save results
        save_summary(s, f'state-{expstr}--{country}-{area}.pk')

2020-03-10 2020-07-31
      Canton      Datum  age_group  days
24        VD 2020-05-14          0    65
25        VD 2020-05-14          1    65
26        VD 2020-05-14          1    65
27        VD 2020-05-14          2    65
28        VD 2020-05-14          2    65
...      ...        ...        ...   ...
75761     VD 2020-04-25          8    46
75769     VD 2020-03-25          8    15
75770     VD 2020-03-30          8    20
75771     VD 2020-04-03          8    24
75772     VD 2020-04-12          8    33

[2898 rows x 4 columns]
(70, 9)
[[ 44  44  44  44  44  44  44  44  44]
 [ 48  48  48  48  48  48  48  48  48]
 [ 52  52  52  52  52  52  52  52  52]
 [ 56  56  56  56  56  56  56  56  56]
 [ 60  60  60  60  60  60  60  60  60]
 [ 64  64  64  64  64  64  64  64  64]
 [ 68  68  68  68  68  68  68  68  68]
 [ 72  72  72  72  72  72  72  72  72]
 [ 76  76  76  76  76  76  76  76  76]
 [ 80  80  80  80  80  80  80  80  80]
 [ 84  84  84  84  84  84  84  84  84]
 [ 88  88  88  88  88  8

AssertionError: 

## 3. Plot the results

Import libs

In [ ]:
from lib.plot import Plotter

Create figures from summaries

In [ ]:
for country, areas in locs.items():
    for area in areas:
        s = load_summary(f'state-{expstr}--{country}-{area}.pk')
        plotter = Plotter()
        titles_SD_4 = ['No curfew'] + [f'K = {k}' for k in K_groups]
        plotter.compare_total_infections(
            summaries_SD_4, 
            titles=titles_SD_4, 
            figtitle=r'Curfew of K random groups for staggered K - 1 day windows',
            filename=runstr + 'SD_050', 
            figsize=(6,4), acc=500, 
            ymax=500, errorevery=14)